In [24]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression

# ---- Params ----
r = 0.02
sigma = 0.15
x0 = 1
T = 12
n_steps = 50
n_paths = 200
seed = 42

# ---- Simulate GBM (exact solution) ----
dt = T / n_steps
times = np.linspace(0, T, n_steps + 1)
rng = np.random.default_rng(seed)
Z = rng.standard_normal(size=(n_paths, n_steps))
dW = Z * np.sqrt(dt)
W = np.cumsum(dW, axis=1)
W = np.concatenate([np.zeros((n_paths, 1)), W], axis=1)
drift = (r - 0.5 * sigma**2) * times
logS = np.log(x0) + drift[np.newaxis, :] + sigma * W
S = np.exp(logS)  # shape (n_paths, n_steps+1)

# ---- Prepare data ----
ST = S[:, -1]  # terminal values
df = pd.DataFrame(S.T, index=times, columns=[f'{i}' for i in range(n_paths)])

# Theoretical log-normal params for S_T
mu_ln = np.log(x0) + (r - 0.5 * sigma**2) * T
sigma_ln = sigma * np.sqrt(T)

# grid for PDF
x_grid = np.linspace(max(1e-8, ST.min() * 0.8), ST.max() * 1.2, 400)
pdf_theo = (1.0 / (x_grid * sigma_ln * np.sqrt(2 * np.pi))) * \
          np.exp(- (np.log(x_grid) - mu_ln)**2 / (2 * sigma_ln**2))

# ---- Create subplot: left = paths, right = marginal histogram + PDF ----
fig = make_subplots(rows=1, cols=2,
                    column_widths=[0.72, 0.28],
                    specs=[[{"type":"xy"}, {"type":"xy"}]])

# ---- Left: plot paths colored by ST ----
for i in ST.argsort():
    fig.add_trace(go.Scatter(x=times, y=S[i, :], mode='lines', line=dict(color=None, width=1), opacity=0.5, showlegend=False), row=1, col=1)

# Empirical mean and theoretical mean
fig.add_trace(go.Scatter(x=times, y=df.mean(axis=1), mode='lines', line=dict(color='black', width=2, dash='dot'), showlegend=False), row=1, col=1)
fig.update_xaxes(title_text='t', row=1, col=1)
fig.update_yaxes(title_text='X<sub>t', row=1, col=1)

# ---- Right: horizontal histogram (density) + theoretical PDF (as line) ----
hist_vals, bin_edges = np.histogram(ST, bins=25, density=True)
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

# Add histogram as horizontal bars (orientation='h')
fig.add_trace(go.Bar(x=hist_vals, y=bin_centers, orientation='h', showlegend=False), row=1, col=2)

# Add theoretical PDF as a line (x=pdf, y=x_grid)
fig.add_trace(go.Scatter(x=pdf_theo, y=x_grid, mode='lines', line=dict(width=2), showlegend=False), row=1, col=2)

# Add vertical line for E[S_T]
E_ST = x0 * np.exp(r * T)
fig.add_trace(go.Scatter(x=[0, max(hist_vals)], y=[E_ST, E_ST], mode='lines', line=dict(color='black', width=2, dash='dot'), showlegend=False), row=1, col=2)

fig.update_yaxes(title_text='X<sub>T', row=1, col=2)
fig.update_xaxes(title_text='Density', row=1, col=2)
fig.update_layout(title_text=f"Geometric Brownian Motion - {n_paths} simulations - {n_steps} steps - {int(T)} months")

fig.show()

# ---- Params for American Call ----
K = x0  # strike price

# ---- Calculate American Call payoff for each path and time ----
payoffs = np.maximum(S - K, 0)  # shape (n_paths, n_steps+1)
payoff_final = payoffs.max(axis=1)  # max payoff over time for each path

# ---- Create subplot: 1 row, 2 cols ----
fig_payoff = make_subplots(rows=1, cols=2,
                           column_widths=[0.72, 0.28],
                           specs=[[{"type":"xy"}, {"type":"xy"}]])

# ---- Left: plot payoff paths ----
for i in payoff_final.argsort():
    fig_payoff.add_trace(go.Scatter(x=times, y=payoffs[i, :], mode='lines', line=dict(color=None, width=1), opacity=0.5, showlegend=False), row=1, col=1)
fig_payoff.add_trace(go.Scatter(x=times, y=payoffs.mean(axis=0), mode='lines', line=dict(color='red', width=2, dash='dot'), showlegend=False), row=1, col=1)
fig_payoff.update_xaxes(title_text='t', row=1, col=1)
fig_payoff.update_yaxes(title_text='Payoff', row=1, col=1)

# ---- Right: histogram of final payoffs ----
hist_payoff_vals, bin_payoff_edges = np.histogram(payoff_final, bins=25, density=True)
bin_payoff_centers = 0.5 * (bin_payoff_edges[:-1] + bin_payoff_edges[1:])
fig_payoff.add_trace(go.Bar(x=hist_payoff_vals, y=bin_payoff_centers, orientation='h', showlegend=False), row=1, col=2)
fig_payoff.update_yaxes(title_text='Final Payoff', row=1, col=2)
fig_payoff.update_xaxes(title_text='Density', row=1, col=2)

fig_payoff.update_layout(title_text=f"American Call Payoff - {n_paths} simulations - {n_steps} steps - {int(T)} months<br>  K = {K}")

fig_payoff.show()

X, Y = [], []

for i in range(1, len(df)):
    for j in range(len(df.iloc[i])):
        X.append(df.iloc[i-1,j])
        Y.append(max(df.iloc[i,j] - K, 0) * np.exp(- r * dt))

m = min(X)

figg = go.Figure()
figg.add_trace(go.Scatter(x=X, y=Y, mode='markers', name='Empyrical Values'))
for i in range(len(Y)-1, -1, -1):
    if Y[i] == 0:
        Y.pop(i)
        X.pop(i)
X = np.array(X).reshape(-1, 1)  # sklearn attend un array 2D pour X
Y = np.array(Y)
model = LinearRegression()
model.fit(X, Y)
a, b = model.intercept_, model.coef_[0]
print(-a/b)
figg.add_trace(go.Scatter(x=[m, -a/b], y=[0, 0], mode='lines', showlegend=False))
figg.add_trace(go.Scatter(x=[-a/b, max(X)[0]], y=[0, max(X)[0]*b+a], mode='lines', name='Linear Regression'))

figg.update_xaxes(title_text='X<sub>t<sub>n-1')
figg.update_yaxes(title_text='V(t<sub>n-1</sub> , t<sub>n</sub>)')
figg.update_layout(title='Longstaff-Schwartz Algorithm - Continuation values')
figg.show()

0.9743526215745443


In [25]:
def continuation_value(x):
    return a + b * x

n_times = len(df)
n_paths = df.shape[1]

exercise_times = np.full(n_paths, n_times-1)  # par défaut exercice à maturité
exercise_values = np.zeros(n_paths)

# Pour chaque path, cherche le premier temps d'exercice anticipé
for j in range(n_paths):
    for i in range(n_times-1):  # on ne regarde pas le dernier point (maturité)
        x_t = df.iloc[i, j]
        payoff_immed = max(x_t - K, 0)
        val_cont = continuation_value(x_t)
        if payoff_immed > 0 and payoff_immed > val_cont:
            exercise_times[j] = i
            exercise_values[j] = payoff_immed
            break
    else:
        # Si jamais pas d'exercice anticipé, payoff à maturité
        exercise_values[j] = max(df.iloc[-1, j] - K, 0)

# Maintenant on trace les trajectoires tronquées
fig = go.Figure()

times = df.index.values

for j in range(n_paths):
    t_stop = exercise_times[j]
    fig.add_trace(go.Scatter(
        x=times[:t_stop+1],
        y=df.iloc[:t_stop+1, j],
        mode='lines',
        line=dict(width=1),
        opacity=0.5,
        showlegend=False
    ))
    # Marqueur à l'exercice
    fig.add_trace(go.Scatter(
        x=[times[t_stop]],
        y=[df.iloc[t_stop, j]],
        mode='markers',
        marker=dict(color='black', size=5),
        showlegend=False
    ))

fig.update_layout(
    title="Early Exercise Points",
    xaxis_title="t",
    yaxis_title="X<sub>t"
)

fig.show()

payoffs_discounted = np.array([
    exercise_values[j] * np.exp(-r * dt * exercise_times[j])
    for j in range(n_paths)
])
call_american_price = np.mean(payoffs_discounted)
print(f"Estimated American Call Price: {call_american_price:.4f}")

Estimated American Call Price: 0.2816
